In [2]:
import networkx as nx
import numpy as np
import random
from src.diffusion_models import independent_cascade_model, linear_threshold_model
from src.polarization import optimize_tau 
from src.graph import random_color_graph, create_polarized_graph
from src.seed import seed_random, seed_degree

random.seed(42)

In [3]:
def get_red_and_blue_nodes(G):
    red_nodes = [node for node in G.nodes if G.nodes[node]["color"] == "red"]
    blue_nodes = [node for node in G.nodes if G.nodes[node]["color"] == "blue"]
    return red_nodes, blue_nodes

def add_edges_using_optimize_tau(G, C, k, perc_k):
    """Add edges to the graph G using the optimize_tau function."""
    red_nodes, blue_nodes = get_red_and_blue_nodes(G)
    F_set, _ = optimize_tau(C, G, k, red_nodes, blue_nodes, perc_k)
    G.add_edges_from(F_set)

def add_random_edges(G, C, k):
    """Add the same number of random edges as the optimize_tau function would add."""

    potential_edges = list(nx.non_edges(G))
    random_edges = random.sample(potential_edges, k)
    G.add_edges_from(random_edges)

def run_diffusion_with_same_seed(
    create_graph_func,
    num_nodes,
    intra_group_connectness,
    inter_group_connectness,
    add_edges_func,
    diffusion_model_func,
    seed_func,
    seed_count,
    threshold=None,
    k=5,
    perc_k=5,
    random_seed=42,
):
    # Create original graph
    original_graph = create_graph_func(
        num_nodes, intra_group_connectness, inter_group_connectness
    )
    # color the graph
    random_color_graph(original_graph)

    # Select seeds
    seeds = seed_func(original_graph, seed_count, random_seed)

    # Generate thresholds for each node if using linear threshold model
    if diffusion_model_func == linear_threshold_model:
        thresholds = {node: np.random.uniform(0, 1) for node in original_graph.nodes}
    else:
        thresholds = threshold

    # Add new edges to the graph using the optimized method
    modified_graph_opt = original_graph.copy()
    add_edges_using_optimize_tau(modified_graph_opt, C=seeds, k=k, perc_k=perc_k)

    # Run diffusion on original graph
    result_original = diffusion_model_func(original_graph, seeds, thresholds, random_seed)

    # Run diffusion on optimized modified graph
    result_modified_opt = diffusion_model_func(modified_graph_opt, seeds, thresholds, random_seed)

    # Add new edges to the graph using random edges method
    modified_graph_rand = original_graph.copy()
    add_random_edges(modified_graph_rand, C=seeds, k=k)

    # Run diffusion on randomly modified graph
    result_modified_rand = diffusion_model_func(modified_graph_rand, seeds, thresholds, random_seed)

    print("Original graph")
    print("Number of nodes: ", original_graph.number_of_nodes())
    print("Number of edges: ", original_graph.number_of_edges())
    print("Number of activated nodes: ", result_original[0])

    print("Optimized modified graph")
    print("Number of nodes: ", modified_graph_opt.number_of_nodes())
    print("Number of edges: ", modified_graph_opt.number_of_edges())
    print("Number of activated nodes: ", result_modified_opt[0])


    print("Randomly modified graph")
    print("Number of nodes: ", modified_graph_rand.number_of_nodes())
    print("Number of edges: ", modified_graph_rand.number_of_edges())
    print("Number of activated nodes: ", result_modified_rand[0])


    return result_original, result_modified_opt, result_modified_rand

In [6]:
result_original, result_modified, result_modified_rand = run_diffusion_with_same_seed(
    create_polarized_graph,
    num_nodes=2000,
    intra_group_connectness=0.2,
    inter_group_connectness=0.01,
    add_edges_func=add_edges_using_optimize_tau,
    diffusion_model_func=independent_cascade_model,
    seed_func=seed_random,
    seed_count=150,
    threshold=1,
    k=1000,
    perc_k=7,
)
result_original[0], result_modified[0], result_modified_rand[0]

100%|██████████| 1000/1000 [1:14:49<00:00,  4.49s/it]


Original graph
Number of nodes:  2000
Number of edges:  409971
Number of activated nodes:  737
Optimized modified graph
Number of nodes:  2000
Number of edges:  410906
Number of activated nodes:  742
Randomly modified graph
Number of nodes:  2000
Number of edges:  410971
Number of activated nodes:  754


(737, 742, 754)